In [1]:
from json import load
from dotenv import load_dotenv
import os
import pandas as pd
import tensorflow as tf

load_dotenv()

data_path = os.getenv('DATA_PATH')
ambient_dir = f'{data_path}/ambient' 
attack_dir = f'{data_path}/attacks'

df_train = pd.read_parquet(f'{data_path}/ambient/ambient_train_chunks.parquet')

In [2]:
import tensorflow as tf

# Shuffle the DataFrame
df_train = df_train.sample(frac=1).reset_index(drop=True)

# Split into features and labels
X = df_train.drop(columns=['label'])
y = df_train['label']

# Convert to NumPy arrays
X = X.values
y = y.values

# Convert to tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y))

# Specify batch size
batch_size = 32
dataset = dataset.batch(batch_size)

2023-10-20 10:54:22.194547: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-10-20 10:54:22.194569: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-20 10:54:22.194573: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-20 10:54:22.194606: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 10:54:22.194622: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


706499

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model


N = 11
# Parameters
EMBEDDING_DIM = 50    # Embedding dimension for AID
AID_COUNT = df_train['aid'].nunique()  # Number of unique AIDs in your data

# Model definition
input_time_diffs = layers.Input(shape=(N,), name="input_time_diffs")
dense1 = layers.Dense(128, activation='relu')(input_time_diffs)
dense2 = layers.Dense(64, activation='relu')(dense1)
output = layers.Dense(1)(dense2)   # Regression problem, so no activation in the output layer

model = Model(inputs=input_time_diffs, outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_time_diffs (InputLay  [(None, 11)]              0         
 er)                                                             
                                                                 
 dense (Dense)               (None, 128)               1536      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9857 (38.50 KB)
Trainable params: 9857 (38.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
del df_train
del X
del y

In [6]:
# Number of epochs
num_epochs = 1

# Training the model using the dataset
model.fit(dataset, epochs=num_epochs)

: 